# My Solution for Titanic Competition  

Thanks to "A Data Science Framework:To Achieve 99% Accuracy" on Kaggle. A fantastic tutorial! 

1. Define the Problem 
2. Gather the Data
3. Prepare Data for Consumption
4. Perform Exploratory Analysis
5. Model Data
6. Validate and Implement Data Model
7. Optimize and Strategize

### 1. Define the problem: 
Develop an algorithm to predict the survival outcome of passengers on the Titanic.  
It's a Binary classification problem and python is used in this solution.

### 2. Gather the data
The dataset is provided by kaggle with train and test data.

### 3. Prepare data for consumption

#### 3.1 Import Libraries
The following code is written in Python 3.6.

In [1]:
#load packages
import sys
print(f'Python version: {sys.version}')

import pandas as pd
print(f'pandas version: {pd.__version__}')

import matplotlib
print(f'matplotlib version: {matplotlib.__version__}')

import numpy as np
print(f'numpy version: {pd.__version__}')

import scipy as sp
print(f'scipy version: {sp.__version__}')

import IPython
from IPython import display # pretty printing of dataframes in Jupyter notebook
print(f'IPython version: {IPython.__version__}')

import sklearn #collection of machine learning algorithms
print(f'scikit-learn version: {sklearn.__version__}')

import random
import time

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
pandas version: 0.22.0
matplotlib version: 2.1.2
numpy version: 0.22.0
scipy version: 1.0.0
IPython version: 6.2.1
scikit-learn version: 0.19.1
-------------------------


In [24]:
#input data files are available in the "./input/" directory
from subprocess import check_output
print('Input data files:')
#print(check_output(['ls','input']).decode('utf8'))
#print(check_output(["ls","./input"]).decode("utf8"))

Input data files:


#### Load data modeling libraries
I will use the scikit-learn library to develpe my machine learning algorithms.

In [4]:
#common Modeling algorithms
from sklearn import svm,tree,linear_model,neighbors,naive_bayes,ensemble,discriminant_analysis,gaussian_process
from xgboost import XGBClassifier

#common Modeling helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

In [9]:
#configure visualization defaults
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

#### 3.2  Meet the data
Here I will first load in the data, use the info() and sample() function to get a quick and dirty overview of variable datatypes(i.e. qualitative vs quantitative)  


In [11]:
#load the data
#this is the raw dataset, I will split the train set into train and test data in following 
data_raw = pd.read_csv('./input/train.csv')

#the dataset provided is the validtion file for competition submission
data_val = pd.read_csv('./input/test.csv')

#make a copy of original data to play around
data1 = data_raw.copy(deep = True)  #deep = true is default, modifications will not affect the original object

#put two datasets together for preprocessing
data_total = [data1,data_val]

#preview data
print(data_raw.info())

data_raw.sample(10) #see 10 random samples in dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
133,134,1,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.0000,NaN,S
628,629,0,3,"Bostandyeff, Mr. Guentcho",male,26.0,0,0,349224,7.8958,NaN,S
708,709,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,113781,151.5500,NaN,S
777,778,1,3,"Emanuel, Miss. Virginia Ethel",female,5.0,0,0,364516,12.4750,NaN,S
851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,347060,7.7750,NaN,S
742,743,1,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21.0,2,2,PC 17608,262.3750,B57 B59 B63 B66,C
456,457,0,1,"Millet, Mr. Francis Davis",male,65.0,0,0,13509,26.5500,E38,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.2500,NaN,S
370,371,1,1,"Harder, Mr. George Achilles",male,25.0,1,0,11765,55.4417,E50,C
787,788,0,3,"Rice, Master. George Hugh",male,8.0,4,1,382652,29.1250,NaN,Q


#### Variables summary
1. The **Survived** varibale is our outcome or dependent variable(or lable). 